In [110]:
from functions import *
import numpy as np
import pandas as pd
from math import isnan

In [111]:
# Import Guradian data
guardian_path = '/Users/ethan/OneDrive - University of Surrey/Coursework/Final Year/FYP/Data/Guardian/Guardian_University_Guide_2021.xlsx'
guardian = pd.read_excel(guardian_path, sheet_name='Institution', index_col='Name of Provider')
guardian.rename(columns={'rank2020':'Ranking'}, inplace=True)

In [112]:
# Import nss data
nss_path = '/Users/ethan/OneDrive - University of Surrey/Coursework/Final Year/FYP/Data/NSS/NSS_taught_FT20_CAH.xlsx'
nss = pd.read_excel(nss_path, sheet_name='1')

In [113]:
# Sort and merge nss data to be useful
columns = ['Provider','Question Number', 'Subject', 'Actual value', 'Response']
nss = nss[columns]
nss.set_index('Provider', inplace=True)
nss.head()

,Question Number,Subject,Actual value,Response
Provider,,,,
University of Aberdeen,Q01,Medicine and dentistry,0.9900,100
University of Aberdeen,Q02,Medicine and dentistry,0.9600,100
University of Aberdeen,Q03,Medicine and dentistry,1.0000,101
University of Aberdeen,Scale03,Medicine and dentistry,0.7376,101
University of Aberdeen,Q27,Medicine and dentistry,0.9901,101


In [114]:
unis = nss.index.unique()
subs = {}
for uni in unis:
    subjects = list(nss.loc[uni,'Subject'].unique())
    subs[uni] = subjects

agg_func = {'Actual value':'mean', 'Response':'sum'}

grouped = nss.groupby(['Provider','Question Number']).agg(agg_func)
grouped.reset_index(inplace=True)
grouped["Question Number"] = grouped["Question Number"].replace("Q01", "Set 1").replace("Q02", "Set 1").replace("Q03", "Set 1").replace("Scale03", "Set 2").replace("Q27", "Set 3")
grouped.set_index(['Provider','Question Number'], inplace=True)

agg_func = {'Actual value':'mean', 'Response':'mean'}
grouped = grouped.groupby(['Provider','Question Number']).agg(agg_func)

nss_df = pd.DataFrame(round(grouped.xs('Set 1', level=1)['Actual value'], 3))
response = grouped.xs('Set 1', level=1)['Response']
grouped = grouped['Actual value']
nss_df['% Satisfied with Assessment'] = round(grouped.xs('Set 2', level=1), 3)
nss_df['% Satisfied with Course'] = round(grouped.xs('Set 3', level=1), 3)
nss_df.rename(columns={'Actual value':'% Satisfied with Teaching'}, inplace=True)
nss_df = nss_df * 100
nss_df['Response'] = round(response, 0)
nss_df['Subjects'] = nss_df.index.map(subs)
nss_df = nss_df.loc[nss_df['Response'] > 100]
columns = ['% Satisfied with Teaching','% Satisfied with Course','% Satisfied with Assessment', 'Subjects']

nss_df = nss_df[columns]
nss_df[:20]
nss_df.describe()
nss_df.head()

,% Satisfied with Teaching,% Satisfied with Course,% Satisfied with Assessment,Subjects
Provider,,,,
ACM Guildford Limited,73.7,60.3,54.9,"[Design, and creative and performing arts]"
ACM London Limited,58.4,36.6,60.2,"[Design, and creative and performing arts]"
Abertay University,92.3,90.0,82.9,"[Subjects allied to medicine, Biological and s..."
Aberystwyth University,90.0,88.7,81.7,"[Biological and sport sciences, Psychology, Ve..."
Anglia Ruskin University Higher Education Corporation,86.8,81.5,74.2,"[Subjects allied to medicine, Biological and s..."


In [145]:
# Continuation

cont_path = '/Users/ethan/OneDrive - University of Surrey/Coursework/Final Year/FYP/Data/HESA/HESA continuation.csv'
cont = pd.read_csv(cont_path, index_col='HE provider')['Percentage no longer in HE (%)']
cont.sort_index()
cont = pd.DataFrame(cont[:-5])
cont['Continuation %'] = 100 - cont['Percentage no longer in HE (%)']
final_df = pd.DataFrame(cont['Continuation %'])
final_df.head()

275
AA School of Architecture
The University of Aberdeen
Abertay University
Aberystwyth University
ACM Guildford Limited
Academy of Live and Recorded Arts
AECC University College
Amity Global Education
Anglia Ruskin University
Apex College London
Arden University
Arts Educational Schools
Hult International Business School Ltd
Aston University
Bangor University
Bath Spa University
The University of Bath
University of Bedfordshire
Queen's University Belfast
BIMM Limited
Birkbeck College
Birmingham City University
The University of Birmingham
University College Birmingham
Bristol Baptist College
Bishop Grosseteste University
Bloomsbury Institute
The University of Bolton
The Arts University Bournemouth
Bournemouth University
BPP University
The University of Bradford
The University of Brighton
The University College of Osteopathy
The University of Bristol
Brit College
Trinity College Bristol
Brunel University London
Buckinghamshire New University
The University of Buckingham
UK College of B

KeyError: 'The Univesity of Oxford'

In [116]:
# Grad Salaries

salaries_path = '/Users/ethan/OneDrive - University of Surrey/Coursework/Final Year/FYP/Data/HESA/HESA salaries.csv'
salaries = pd.read_csv(salaries_path, index_col='Provider Name')
salaries.drop(axis=1, labels='UKPRN', inplace=True)
salaries.drop(axis=0, labels='Total', inplace=True)
salaries.replace(',','',regex=True,inplace=True)
salaries = salaries.astype(int)
salaries['Total Graduates'] = salaries.sum(axis=1)

to_salaries = [15000, 16500, 19500, 22500, 25500, 28500, 31500, 34500, 37500, 39000]
for index, col in enumerate(salaries.columns[:-1]):
    salaries[col] = salaries[col] * to_salaries[index]

salaries['Total'] = salaries.iloc[:,:-1].sum(axis=1)
salaries = salaries[['Total', 'Total Graduates']]
salaries = salaries.loc[salaries['Total Graduates'] > 50]
salaries['Average Salary'] = salaries['Total'] / salaries['Total Graduates']
salaries.dropna(axis=0, how='any', inplace=True)
final_df['Average Salary'] = salaries['Average Salary'].astype(int)
final_df[:20]

,Continuation %,Average Salary
HE provider,,
AA School of Architecture,NaN,NaN
The University of Aberdeen,95.6,28007.0
Abertay University,87.9,25000.0
Aberystwyth University,95.9,23466.0
ACM Guildford Limited,83.6,NaN
Academy of Live and Recorded Arts,90.5,NaN
AECC University College,96.0,NaN
Amity Global Education,NaN,NaN
Anglia Ruskin University,89.7,26387.0


In [117]:
# Expenditure Academic Services

student_path = '/Users/ethan/OneDrive - University of Surrey/Coursework/Final Year/FYP/Data/HESA/hesa-student-2021.csv'
student = pd.read_csv(student_path, index_col='HE provider')
AS17_path = '/Users/ethan/OneDrive - University of Surrey/Coursework/Final Year/FYP/Data/HESA/HESA Academic Services 17.csv'
AS17 = pd.read_csv(AS17_path, index_col='HE Provider')
AS18_path = '/Users/ethan/OneDrive - University of Surrey/Coursework/Final Year/FYP/Data/HESA/HESA Academic Services 18.csv'
AS18 = pd.read_csv(AS18_path, index_col='HE Provider')
AS17 = AS17.replace(',', '', regex=True).dropna(axis=1).astype(int)
AS18 = AS18.replace(',', '', regex=True).dropna(axis=1).astype(int)
AS_total = pd.concat((AS17, AS18))
AS_total = AS_total.groupby(AS_total.index).mean()
AS_total.sort_values(by='HE Provider', inplace=True)
AS_total['Students'] = student['Total'].replace(',','',regex=True).astype(int)
AS_total.dropna(axis=0, how='any', subset=['Students'], inplace=True)
AS_total = AS_total.loc[AS_total['Students'] > 500]
AS_exp_per_student = (AS_total['Total expenditure'] / AS_total['Students']) * 1000
AS_exp_per_student = AS_exp_per_student.loc[AS_exp_per_student!=0]
final_df['Academic Services Expenditure per Student'] = AS_exp_per_student.astype(int)
final_df.head()

,Continuation %,Average Salary,Academic Services Expenditure per Student
HE provider,,,
AA School of Architecture,NaN,NaN,NaN
The University of Aberdeen,95.6,28007.0,1507.0
Abertay University,87.9,25000.0,1002.0
Aberystwyth University,95.9,23466.0,1735.0
ACM Guildford Limited,83.6,NaN,NaN


In [144]:
# Expenditure Facilities

AC17_path = '/Users/ethan/OneDrive - University of Surrey/Coursework/Final Year/FYP/Data/HESA/HESA Services 17.csv'
AC17 = pd.read_csv(AC17_path, index_col='HE Provider')
AC18_path = '/Users/ethan/OneDrive - University of Surrey/Coursework/Final Year/FYP/Data/HESA/HESA Services 18.csv'
AC18 = pd.read_csv(AC18_path, index_col='HE Provider')
prem17_path = '/Users/ethan/OneDrive - University of Surrey/Coursework/Final Year/FYP/Data/HESA/HESA Premises 17.csv'
prem17 = pd.read_csv(prem17_path, index_col='HE Provider')
prem18_path = '/Users/ethan/OneDrive - University of Surrey/Coursework/Final Year/FYP/Data/HESA/HESA Premises 18.csv'
prem18 = pd.read_csv(prem18_path, index_col='HE Provider')
AC17 = AC17.replace(',', '', regex=True).dropna(axis=1).astype(int)
AC18 = AC18.replace(',', '', regex=True).dropna(axis=1).astype(int)
AC_total = pd.concat((AC17, AC18))
AC_total = AC_total.groupby(AC_total.index).mean()
AC_total.sort_values(by='HE Provider', inplace=True)

prem17 = prem17.replace(',', '', regex=True).dropna(axis=1).astype(int)
prem18 = prem18.replace(',', '', regex=True).dropna(axis=1).astype(int)
prem_total = pd.concat((AC17, AC18))
prem_total = prem_total.groupby(prem_total.index).mean()

facilities = pd.concat((AC_total['Total expenditure'],prem_total['Total expenditure'], student['Total']),axis=1)
facilities.columns = ['AC_exp', 'prem_exp', 'Students']
facilities = facilities.replace(',','',regex=True).dropna(axis=0,how='any').astype(int)
facilities['Expenditure per Student'] = ((facilities['AC_exp'] + facilities['prem_exp']) / facilities['Students']) * 1000
facilities = facilities.loc[facilities['Expenditure per Student']!=0]
facilities = facilities.loc[facilities['Students'] > 500]
final_df['Facilities Expenditure per Student'] = facilities['Expenditure per Student'].astype(int)
final_df.head()

,Continuation %,Average Salary,Academic Services Expenditure per Student,Facilities Expenditure per Student,Student/Staff Ratio,% Graduates in High Skilled Work,Applications to Acceptance (%),% Satisfied with Teaching,% Satisfied with Course,% Satisfied with Assessment
HE provider,,,,,,,,,,
The University of Aberdeen,95.6,28007.0,1507.0,1651,13.1,39.7,13.270,88.2,86.9,70.8
Abertay University,87.9,25000.0,1002.0,2664,21.4,36.5,20.835,92.3,90.0,82.9
Aberystwyth University,95.9,23466.0,1735.0,1423,15.0,29.0,19.085,90.0,88.7,81.7
Anglia Ruskin University,89.7,26387.0,778.0,1594,35.9,48.8,15.100,86.8,81.5,74.2
Aston University,96.3,26802.0,1177.0,1598,18.2,56.0,14.465,83.3,83.6,69.9


In [142]:
# Student Staff Ratio (all academic staff)

staff_path = '/Users/ethan/OneDrive - University of Surrey/Coursework/Final Year/FYP/Data/HESA/HESA Staff.csv'
staff = pd.read_csv(staff_path, index_col = 'HE Provider')
staff['Students'] = student['Total']
columns = ['Total academic staff (excluding atypical)', 'Students']
staff = staff[columns]
staff.replace(',', '', regex=True, inplace=True)
staff.dropna(axis=0, how='any', inplace=True)
staff = staff.astype(int)
staff['Student/Staff Ratio'] = round(staff['Students'] / staff['Total academic staff (excluding atypical)'], 1)
staff = staff.loc[staff['Total academic staff (excluding atypical)'] > 100]
staff = staff.loc[staff['Students'] > 500]
final_df['Student/Staff Ratio'] = staff['Student/Staff Ratio']
final_df.head()

112
The University of Aberdeen
Abertay University
Aberystwyth University
Anglia Ruskin University
Aston University
Bangor University
Bath Spa University
The University of Bath
University of Bedfordshire
Queen's University Belfast
Birkbeck College
Birmingham City University
The University of Birmingham
Bishop Grosseteste University
The University of Bolton
Bournemouth University
The University of Bradford
The University of Brighton
The University of Bristol
Brunel University London
Buckinghamshire New University
The University of Buckingham
Canterbury Christ Church University
Cardiff University
Cardiff Metropolitan University
The University of Central Lancashire
University of Chester
The University of Chichester
City, University of London
Coventry University
University of Cumbria
De Montfort University
University of Derby
The University of Dundee
University of Durham
The University of East Anglia
The University of East London
Edge Hill University
Edinburgh Napier University
The Universi

KeyError: 'The Univesity of Cambridge'

In [138]:
# Outcomes

outcomes_path = '/Users/ethan/OneDrive - University of Surrey/Coursework/Final Year/FYP/Data/HESA/HESA Outcomes.csv'
detailed_path = '/Users/ethan/OneDrive - University of Surrey/Coursework/Final Year/FYP/Data/HESA/HESA Outcomes Detailed.csv'
outcomes = pd.read_csv(outcomes_path, index_col='Provider Name')
detailed = pd.read_csv(detailed_path, index_col='Provider Name')

detailed['Total'] = outcomes['Total with known outcomes']

outcomes = detailed[['Total high skilled', 'Total']]
outcomes = outcomes.replace(',', '', regex=True)
outcomes = outcomes.astype(int)
outcomes['% in High Skilled Work'] = round(outcomes['Total high skilled'] / outcomes['Total'] * 100, 1)
outcomes = outcomes.loc[outcomes['Total'] > 50]
final_df['% Graduates in High Skilled Work'] = outcomes['% in High Skilled Work']
final_df.head()

KeyError: 'University of Oxford'

In [121]:
# Add acceptance rate averaged over 2 years
acceptances = pd.read_csv('Data/UCAS/EOC_acceptances.csv')
applications = pd.read_csv('Data/UCAS/EOC_applications_main.csv')
applications.set_index(['Provider', 'Year'], inplace=True)
acceptances.set_index(['Provider', 'Year'], inplace=True)
applications['Accepted'] = acceptances['Accepted applicants through the main scheme']
applications.dropna(how='any', axis=0, inplace=True)
applications['Percentage Accepted'] = round((applications['Accepted'] / applications['Main scheme applications']) * 100, 2)
applications = applications.loc[applications['Main scheme applications'] >= 100]
applications.reset_index(inplace=True)
applications = applications.pivot_table(index='Provider', columns='Year', values='Percentage Accepted')
applications = applications[[2018, 2019]]
applications['Acceptance Rate'] = applications.mean(axis=1)
applications.reset_index(inplace=True)

for i in range(len(applications['Provider'])):
    applications.loc[i, 'Provider'] = applications.loc[i, 'Provider'][4:]
applications.set_index('Provider', inplace=True)

try:
    from index_conversions import applications_to_hesa
    applications_to_hesa = applications_to_hesa
except:
    applications_to_hesa = copy_index(final_df, applications)

applications = pd.DataFrame(applications).rename(index=applications_to_hesa)
final_df['Applications to Acceptance (%)'] = applications['Acceptance Rate']
final_df


,Continuation %,Average Salary,Academic Services Expenditure per Student,Facilities Expenditure per Student,Student/Staff Ratio,% Graduates in High Skilled Work,Applications to Acceptance (%)
HE provider,,,,,,,
AA School of Architecture,NaN,NaN,NaN,NaN,NaN,66.7,NaN
The University of Aberdeen,95.6,28007.0,1507.0,1651.0,13.1,39.7,13.270
Abertay University,87.9,25000.0,1002.0,2664.0,21.4,36.5,20.835
Aberystwyth University,95.9,23466.0,1735.0,1423.0,15.0,29.0,19.085
ACM Guildford Limited,83.6,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
The University of Wolverhampton,88.7,25159.0,1368.0,1676.0,21.1,40.7,18.125
University of Worcester,92.5,24032.0,818.0,2365.0,22.1,46.4,21.560
Writtle University College,90.5,NaN,1177.0,12933.0,NaN,31.0,27.755


In [134]:
# Add NSS data

try:
    from index_conversions import guardian_to_hesa
except:
    guardian_to_hesa = copy_index(final_df, nss_df)

nss_df = nss_df.rename(index=guardian_to_hesa )
columns = ['% Satisfied with Teaching', '% Satisfied with Course', '% Satisfied with Assessment', 'Subjects']
final_df[columns] = nss_df
final_df


,Continuation %,Average Salary,Academic Services Expenditure per Student,Facilities Expenditure per Student,Student/Staff Ratio,% Graduates in High Skilled Work,Applications to Acceptance (%),% Satisfied with Teaching,% Satisfied with Course,% Satisfied with Assessment,Subjects
HE provider,,,,,,,,,,,
The University of Aberdeen,95.6,28007.0,1507.0,1651.0,13.1,39.7,13.270,88.2,86.9,70.8,"[Medicine and dentistry, Subjects allied to me..."
Abertay University,87.9,25000.0,1002.0,2664.0,21.4,36.5,20.835,92.3,90.0,82.9,"[Subjects allied to medicine, Biological and s..."
Aberystwyth University,95.9,23466.0,1735.0,1423.0,15.0,29.0,19.085,90.0,88.7,81.7,"[Biological and sport sciences, Psychology, Ve..."
Anglia Ruskin University,89.7,26387.0,778.0,1594.0,35.9,48.8,15.100,86.8,81.5,74.2,"[Subjects allied to medicine, Biological and s..."
Aston University,96.3,26802.0,1177.0,1598.0,18.2,56.0,14.465,83.3,83.6,69.9,"[Subjects allied to medicine, Biological and s..."
...,...,...,...,...,...,...,...,...,...,...,...
The University of Winchester,93.4,24286.0,1132.0,2081.0,18.2,38.7,24.670,84.9,79.2,74.2,"[Biological and sport sciences, Psychology, So..."
The University of Wolverhampton,88.7,25159.0,1368.0,1676.0,21.1,40.7,18.125,83.8,82.6,74.1,"[Subjects allied to medicine, Biological and s..."
University of Worcester,92.5,24032.0,818.0,2365.0,22.1,46.4,21.560,88.6,86.6,80.2,"[Subjects allied to medicine, Biological and s..."


In [135]:
# Remove Unis with more than x nan values and with less than 4 subjects offered and save to a csv file
print(len(final_df))
x = 4
final_df.dropna(thresh=final_df.shape[1]-x, axis=0, inplace=True)
final_df['Subjects'] = final_df['Subjects'].fillna("").apply(list)
final_df.to_csv('league_table.csv')
final_df = final_df.loc[(final_df.Subjects.map(len) > 7)]
final_df.drop(axis=1, columns='Subjects', inplace=True)
final_df[:30]

112


,Continuation %,Average Salary,Academic Services Expenditure per Student,Facilities Expenditure per Student,Student/Staff Ratio,% Graduates in High Skilled Work,Applications to Acceptance (%),% Satisfied with Teaching,% Satisfied with Course,% Satisfied with Assessment
HE provider,,,,,,,,,,
The University of Aberdeen,95.6,28007.0,1507.0,1651.0,13.1,39.7,13.270,88.2,86.9,70.8
Abertay University,87.9,25000.0,1002.0,2664.0,21.4,36.5,20.835,92.3,90.0,82.9
Aberystwyth University,95.9,23466.0,1735.0,1423.0,15.0,29.0,19.085,90.0,88.7,81.7
Anglia Ruskin University,89.7,26387.0,778.0,1594.0,35.9,48.8,15.100,86.8,81.5,74.2
Aston University,96.3,26802.0,1177.0,1598.0,18.2,56.0,14.465,83.3,83.6,69.9
Bangor University,93.5,23318.0,1116.0,1845.0,13.3,31.8,23.530,86.4,86.4,76.4
Bath Spa University,92.7,22584.0,1313.0,1985.0,22.5,36.5,17.725,82.1,79.0,69.3
The University of Bath,98.1,29933.0,1542.0,1906.0,14.3,58.2,12.940,89.6,88.9,67.9
University of Bedfordshire,79.1,24728.0,874.0,1273.0,34.8,42.7,13.535,79.6,73.3,69.9


In [124]:
# Convert to s-score table

s_score_df = final_df.copy()

for col in final_df.columns:
    x = s_score(final_df,col)
    s_score_df[col] = x

s_score_df['Student/Staff Ratio'] = - s_score_df['Student/Staff Ratio']
nonans = s_score_df.copy()
for ind in s_score_df.index:
    nonans.loc[ind] = s_score_df.loc[ind].fillna(value=s_score_df.loc[ind].mean())

In [125]:
# Weights

weights = []

# Uncomment to have interactive weights settings
for col in final_df.columns:
    while True:
        try:
            x = float(input(f'Weight for {col} (out of 10): '))
            if x > 10 or x < 0:
                raise Exception
            weights.append(x)
            break
        except ValueError:
            print('Enter a number between 0 and 10, no letters!')
        except Exception:
            print('Enter a number between 0 and 10!')

weights = [weight / sum(weights) for weight in weights]
print(weights)

Enter a number between 0 and 10, no letters!
Enter a number between 0 and 10, no letters!
Enter a number between 0 and 10, no letters!
Enter a number between 0 and 10, no letters!
Enter a number between 0 and 10, no letters!
Enter a number between 0 and 10, no letters!
Enter a number between 0 and 10, no letters!
Enter a number between 0 and 10, no letters!
Enter a number between 0 and 10, no letters!
Enter a number between 0 and 10, no letters!
Enter a number between 0 and 10, no letters!
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]


In [126]:
weighted = nonans.copy()
for i, col in enumerate(nonans.columns):
    weighted[col] = nonans[col] * weights[i]

weighted['Score'] = weighted[list(weighted.columns)].sum(axis=1)
weighted.sort_values(by='Score', ascending=False, inplace=True)
weighted['Rank'] = [x for x in range(1, len(weighted)+1)]
weighted[:30]

,Continuation %,Average Salary,Academic Services Expenditure per Student,Facilities Expenditure per Student,Student/Staff Ratio,% Graduates in High Skilled Work,Applications to Acceptance (%),% Satisfied with Teaching,% Satisfied with Course,% Satisfied with Assessment,Score,Rank
HE provider,,,,,,,,,,,,
"Imperial College of Science, Technology and Medicine",0.133772,0.364816,0.338172,0.302562,0.190631,0.068948,0.104857,-0.067558,-0.098086,-0.289545,1.048570,1
The University of St Andrews,0.126131,0.092644,0.299721,-0.024999,0.129459,-0.145138,-0.165165,0.259824,0.240160,0.132290,0.944927,2
University of Durham,0.133772,0.111736,0.173882,0.059102,0.096738,0.053755,0.077438,0.019298,0.080841,-0.032187,0.774376,3
University College London,0.110848,0.271819,0.173416,0.272143,0.177828,0.067567,-0.116821,-0.047515,-0.022103,-0.113458,0.773724,4
The University of Bath,0.146508,0.205782,0.038721,-0.012368,0.056905,0.194637,-0.081634,0.129539,0.142117,-0.096043,0.724166,5
Loughborough University,0.128678,0.117401,0.055034,-0.098995,0.078244,0.193256,-0.095199,0.082770,0.154373,0.049084,0.664647,6
Bishop Grosseteste University,-0.047077,-0.167411,-0.017207,0.198673,-0.048369,0.049611,0.450357,-0.010768,0.046526,0.136160,0.590495,7
The University of Southampton,0.133772,0.066995,0.133101,0.002789,0.129459,0.121433,-0.135995,0.042683,0.068586,-0.028317,0.534506,8
The University of Lancaster,0.100659,0.025716,0.167357,-0.017525,0.106697,-0.019449,0.051809,0.072748,0.102900,-0.072822,0.518090,9


# 